# Data Quality Checks for Retail Transactions Using Great Expectations
This notebook implements data quality checks on the **Online Retail Dataset** using **Great Expectations** to ensure clean and reliable data for analysis.

In [10]:
import pandas as pd
import great_expectations as gx

# Initialize an ephemeral data context
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

# Load the Online Retail dataset
data_path = "online_retail.xlsx"
df = pd.read_excel(data_path)

# Add a Pandas Data Source
data_source = context.data_sources.add_pandas(name="online_retail")

# Add a Data Asset to the Data Source
data_asset = data_source.add_dataframe_asset(name="online_retail_asset")

# Add a Batch Definition for the entire DataFrame
batch_definition_name = "online_retail_batch"
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)
assert batch_definition.name == batch_definition_name

# Define Batch Parameters
batch_parameters = {"dataframe": df}

# Retrieve the Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

# Create an Expectation Suite
expectation_suite_name = "online_retail_suite"
suite = gx.ExpectationSuite(name=expectation_suite_name)

# Add Expectations
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="InvoiceNo")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="InvoiceNo")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="StockCode")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="Description")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(column="Quantity", type_="int")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(column="Quantity", min_value=1, max_value=1000)
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeOfType(column="UnitPrice", type_="float")
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(column="UnitPrice", min_value=0.0, max_value=1000.0)
)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(column="Country", value_set=["United Kingdom", "Germany", "France", "Netherlands"])
)

# Add the Expectation Suite to the Context
context.suites.add(suite)

# Validate the Data Against the Suite
validation_results = batch.validate(suite)

# Print Validation Results
print("Validation Results:")
print(validation_results)

# Save Validation Results to a File
with open("validation_results.json", "w") as f:
    import json
    json.dump(validation_results.to_json_dict(), f, indent=4)



Calculating Metrics:   0%|          | 0/42 [00:00<?, ?it/s]

Validation Results:
{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "online_retail-online_retail_asset",
          "column": "InvoiceNo"
        },
        "meta": {},
        "id": "82cd1d61-d8ff-4a0b-9ebf-7fb8d1b23290"
      },
      "result": {
        "element_count": 541909,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": [],
        "partial_unexpected_index_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "online_retail-online_retail_asset",
          "column

### Interpretation of Validation Results

The validation report indicates that 8 expectations were evaluated, with 4 passing and 4 failing. Below is a detailed analysis of the results:

---

#### **Successful Expectations**

1. **InvoiceNo Values Are Not Null**
   - **Result**: Passed
   - **Details**: All `InvoiceNo` values are present (541,909 rows, 0 null values).

2. **StockCode Values Are Not Null**
   - **Result**: Passed
   - **Details**: All `StockCode` values are present.

3. **Quantity Column Has Correct Data Type**
   - **Result**: Passed
   - **Details**: The column type is `int64`, which matches the expected type.

4. **UnitPrice Column Has Correct Data Type**
   - **Result**: Passed
   - **Details**: The column type is `float64`, as expected.

---

#### **Failed Expectations**

1. **Description Values Are Not Null**
   - **Result**: Failed
   - **Details**: 1,454 null values were found, accounting for 0.27% of the rows.
   - **Next Step**: Handle missing values by either removing these rows or imputing appropriate values.

2. **Quantity Values Are Between 1 and 1000**
   - **Result**: Failed
   - **Details**: 10,740 values (1.98%) fall outside the range. Common violations include negative quantities like `-1`, `-12`, and `-24`.
   - **Next Step**: Investigate why negative quantities exist (e.g., cancellations) and correct or exclude these rows.

3. **UnitPrice Values Are Between 0.0 and 1000.0**
   - **Result**: Failed
   - **Details**: 122 values (0.02%) exceed the maximum of 1000. For example, values like `13541.33` and `16888.02` were observed.
   - **Next Step**: Investigate outliers to determine if they are valid or require adjustments.

4. **Country Values Are Within a Defined Set**
   - **Result**: Failed
   - **Details**: 26,008 rows (4.80%) contain unexpected country names like "Australia" and "Norway".
   - **Next Step**: Either expand the allowed set of countries or filter these rows as required.

---

### Suggested Next Steps

1. **Address Null Values**
   - For `Description`, either:
     - Remove rows with null values.
     - Impute missing values using the most common description or a placeholder.

2. **Investigate Negative Quantities**
   - Negative quantities might indicate cancellations. Ensure these rows are treated separately or excluded from analytics.

3. **Handle Price Outliers**
   - Check whether values above 1000 are valid (e.g., bulk orders or pricing errors). Correct or exclude invalid rows.

4. **Review Unexpected Countries**
   - Verify if customers from countries like "Australia" and "Norway" should be included. If valid, update the `value_set` to include these countries.